In [3]:
import numpy as np
import pandas as pd
import math
import pickle

from tqdm.notebook import trange, tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score

In [31]:
DATAFRAME_PATH = './data/labeled-comments.csv'
TF_QUANTITY = 100
GENERATE_TRAIN_TEST = False
MODELS_PATH = './data/models'

def get_vocabulary(df):
    count_vectorizer = CountVectorizer(lowercase=False, stop_words=[])
    cv_fit = count_vectorizer.fit_transform(df['content'])
    word_list = count_vectorizer.get_feature_names()
    frequecy_array = cv_fit.toarray()
    count_list = frequecy_array.sum(axis=0)
    vocabulary = (dict(zip(word_list, count_list)))
    return vocabulary, frequecy_array, word_list


def get_bigrams(df):
    count_vectorizer = CountVectorizer(
        lowercase=False, stop_words=[], ngram_range=(2, 2))
    cv_fit = count_vectorizer.fit_transform(df['content'])
    word_list = count_vectorizer.get_feature_names()
    frequecy_array = cv_fit.toarray()
    count_list = frequecy_array.sum(axis=0)
    vocabulary = (dict(zip(word_list, count_list)))
    return vocabulary, frequecy_array, word_list


def get_doc(df, chosen_words):
    return df['content'].apply(lambda y: ' '.join(
        [x for x in y.split() if x in chosen_words]))


def get_bigram_doc(df, chosen_words):
    def select_only_relevant_bigrams(text):
        bigrams_in_text = [b for l in [text]
                           for b in zip(l.split(" ")[:-1], l.split(" ")[1:])]
        return ' '.join([' '.join(w) for w in bigrams_in_text if ' '.join(w) in chosen_words])
    return df['content'].apply(select_only_relevant_bigrams)


def get_relevant_words(df):
    return list(df.sort_values(
        by=['diff'], ascending=False)['term'])

def select_shared_terms(voc1, voc2):
    # select words that are in bolth vocabularies
    list_sexist_sorted_terms = []
    for key, value in sorted(voc1.items(), key=lambda item: item[1]):
        list_sexist_sorted_terms.append(key)

    shared = []

    for word in list_sexist_sorted_terms:
        if word in voc2.keys():
            shared.append(word) 
    return shared

def select_word_freq(shared, sexist, not_sexist):
    freq = {
        'term': [],
        'sexist-freq': [],
        'not-sexist-freq': [],
        'diff': []
    }
    for term in shared:
        freq['term'].append(term)
        if term in sexist.keys():
            freq['sexist-freq'].append(sexist[term])
        else:
            freq['sexist-freq'].append(0)

        if term in not_sexist.keys():
            freq['not-sexist-freq'].append(not_sexist[term])
        else:
            freq['not-sexist-freq'].append(0)

    freq['diff'] = sexist[term] - not_sexist[term]
    freq = pd.DataFrame(freq)

    # normalizind frequencies
    sum_sexist = sum(freq['sexist-freq'])
    freq['sexist-freq'] = freq['sexist-freq'].apply(
        lambda x: x/sum_sexist)
    sum_not_sexist = sum(freq['not-sexist-freq'])
    freq['not-sexist-freq'] = freq['not-sexist-freq'].apply(
        lambda x: x/sum_not_sexist)
    freq['diff'] = freq['sexist-freq'] - \
        freq['not-sexist-freq']

    sexist_terms = freq[freq['diff'] > 0]
    not_sexist_terms = freq[freq['diff'] < 0]

    # most relevant terms to sexist comments
    sexist_ = sexist_terms.sort_values(
        by='diff', ascending=False)

    # most relevant terms to not sexist comments
    not_sexist_ = not_sexist_terms.sort_values(
        by='diff', ascending=True)

    return sexist_, not_sexist_

def generate_cross_validation_train_and_test(df):
    train_size = math.floor(df.shape[0] * 0.9)
    test_size = math.ceil(df.shape[0] * 0.1)
    for i in range (0,10):
        dataframe = df.sample(frac=1)
        train = dataframe.iloc[:train_size]
        test = dataframe.iloc[train_size:]

        # ------ select most relevant unigrams and bigrams to sexist and not sexist context
        sexist_comments = dataframe[dataframe['avg'] > 0.5]
        sexist_vocabulary, sexist_frequency_array, sexist_word_list = get_vocabulary(sexist_comments)
        sexist_bigrams, sexist_frequency_array, sexist_word_list = get_bigrams(sexist_comments)

        not_sexist_comments = dataframe[dataframe['avg'] < 0.5]    
        not_sexist_vocabulary, not_sexist_frequency_array, not_sexist_word_list = get_vocabulary(not_sexist_comments)
        not_sexist_bigrams, not_sexist_frequency_array, not_sexist_word_list = get_bigrams(not_sexist_comments)

        shared_unigrams = select_shared_terms(sexist_vocabulary, not_sexist_vocabulary)
        sexist_unigrams, not_sexist_unigrams = select_word_freq(shared_unigrams, sexist_vocabulary, not_sexist_vocabulary)

        shared_bigrams = select_shared_terms(sexist_bigrams, not_sexist_bigrams)
        sexist_bigrams, not_sexist_bigrams = select_word_freq(shared_bigrams, sexist_bigrams, not_sexist_bigrams)

        # ------ calculate and serialize term frequency to sexist and not sexist unigrams and bigrams
        sexist_vectorizer = TfidfVectorizer(
            stop_words=[],
            use_idf=False,
            norm=None,
            decode_error='replace',
            max_features=TF_QUANTITY,
        )
        not_sexist_vectorizer = TfidfVectorizer(
            stop_words=[],
            use_idf=False,
            decode_error='replace',
            max_features=TF_QUANTITY,
        )
        sexist_bigrams_vectorizer = TfidfVectorizer(
            stop_words=[],
            use_idf=False,
            ngram_range=(2, 2),
            decode_error='replace',
            max_features=TF_QUANTITY,
        )
        not_sexist_bigrams_vectorizer = TfidfVectorizer(
            stop_words=[],
            use_idf=False,
            ngram_range=(2, 2),
            decode_error='replace',
            max_features=TF_QUANTITY,
        )

        relevant_sexist_words = get_relevant_words(sexist_unigrams)
        relevant_not_sexist_words = get_relevant_words(
            not_sexist_unigrams)

        relevant_sexist_bigrams = get_relevant_words(sexist_bigrams)
        relevant_not_sexist_bigrams = get_relevant_words(
            not_sexist_bigrams)

        # ------ sexist unigrams TF
        sexist_doc = get_doc(sexist_comments, relevant_sexist_words)
        not_sexist_doc = get_doc(not_sexist_comments, relevant_sexist_words)
        sexist_tf = pd.DataFrame(sexist_vectorizer.fit_transform(sexist_doc).toarray())
        not_sexist_tf = pd.DataFrame(not_sexist_vectorizer.fit_transform(not_sexist_doc).toarray())
        tf_sexist_dataframe = pd.concat([sexist_tf, not_sexist_tf]).fillna(0)
        tf_sexist_dataframe.columns = [f'TFus_{i}' for i in range(100)]

        # ------ sexist bigrams TF
        sexist_doc = get_bigram_doc(sexist_comments, relevant_sexist_bigrams)
        not_sexist_doc = get_bigram_doc(not_sexist_comments, relevant_sexist_bigrams)
        sexist_bigrams_tf = pd.DataFrame(sexist_bigrams_vectorizer.fit_transform(sexist_doc).toarray())
        not_sexist_bigrams_tf = pd.DataFrame(not_sexist_bigrams_vectorizer.fit_transform(not_sexist_doc).toarray())
        tf_sexist_bigrams_dataframe = pd.concat([sexist_bigrams_tf, not_sexist_bigrams_tf]).fillna(0)
        tf_sexist_bigrams_dataframe.columns = [f'TFbs_{i}' for i in range(100)]

        # ------ not sexist unigrams TF
        sexist_doc = get_doc(sexist_comments, relevant_not_sexist_words)
        not_sexist_doc = get_doc(not_sexist_comments, relevant_not_sexist_words)
        sexist_tf = pd.DataFrame(sexist_vectorizer.fit_transform(sexist_doc).toarray())
        not_sexist_tf = pd.DataFrame(not_sexist_vectorizer.fit_transform(not_sexist_doc).toarray())
        tf_not_sexist_dataframe = pd.concat([sexist_tf, not_sexist_tf]).fillna(0)
        tf_not_sexist_dataframe.columns = [f'TFun_{i}' for i in range(100)]

        # ------ not sexist bigrams TF
        sexist_doc = get_bigram_doc(sexist_comments, relevant_not_sexist_bigrams)
        not_sexist_doc = get_bigram_doc(not_sexist_comments, relevant_not_sexist_bigrams)
        sexist_bigrams_tf = pd.DataFrame(sexist_bigrams_vectorizer.fit_transform(sexist_doc).toarray())
        not_sexist_bigrams_tf = pd.DataFrame(not_sexist_bigrams_vectorizer.fit_transform(not_sexist_doc).toarray())
        tf_not_sexist_bigrams_dataframe = pd.concat([sexist_bigrams_tf, not_sexist_bigrams_tf]).fillna(0)
        tf_not_sexist_bigrams_dataframe.columns = [f'TFbn_{i}' for i in range(100)]

        tf_dataframe = pd.concat([tf_sexist_dataframe,
                                  tf_not_sexist_dataframe,
                                  tf_sexist_bigrams_dataframe,
                                  tf_not_sexist_bigrams_dataframe], axis=1)

        # ------ define quantitative features to train
        likes_df = np.array(pd.concat([sexist_comments['likes'], not_sexist_comments['likes']]).fillna(0))
        dislikes_df = np.array(pd.concat([sexist_comments['dislikes'], not_sexist_comments['dislikes']]).fillna(0))
        char_qty_df = np.array(pd.concat([sexist_comments['char-qty'], not_sexist_comments['char-qty']]).fillna(0))
        word_qty_df = np.array(pd.concat([sexist_comments['word-qty'], not_sexist_comments['word-qty']]).fillna(0))
        sexist_y = sexist_comments['avg'].apply(lambda x: 1)
        not_sexist_y = not_sexist_comments['avg'].apply(lambda x: 0)
        y_df = np.array(pd.concat([sexist_y, not_sexist_y]))

        X_train = tf_dataframe
        X_train['likes'] = likes_df
        X_train['dislikes'] = dislikes_df
        X_train['char-qty'] = char_qty_df
        X_train['word-qty'] = word_qty_df
        X_train['sexist'] = y_df
        X_train = X_train.fillna(0)
        X_train = X_train.sample(frac=1)
        X_train.to_csv(f'./data/{i+1}_train.csv', index=False)

        # define features to test

        doc_vectorizer = TfidfVectorizer(
            stop_words=[],
            use_idf=False,
            norm=None,
            decode_error='replace',
            max_features=TF_QUANTITY,
        )    

        doc = get_doc(test, relevant_sexist_words)
        tf = pd.DataFrame(doc_vectorizer.fit_transform(doc).toarray(), columns=[f'TFus_{i}' for i in range(100)])
        tf_sexist_dataframe = pd.concat([tf]).fillna(0)

        doc = get_bigram_doc(test, relevant_sexist_bigrams)
        tf = pd.DataFrame(doc_vectorizer.fit_transform(doc).toarray(), columns=[f'TFbs_{i}' for i in range(100)])
        tf_sexist_bigrams_dataframe = pd.concat([tf]).fillna(0)

        doc = get_doc(test, relevant_not_sexist_words)
        tf = pd.DataFrame(doc_vectorizer.fit_transform(doc).toarray(), columns=[f'TFun_{i}' for i in range(100)])
        tf_not_sexist_dataframe = pd.concat([tf]).fillna(0)

        doc = get_bigram_doc(test, relevant_not_sexist_bigrams)
        tf = pd.DataFrame(doc_vectorizer.fit_transform(doc).toarray(), columns=[f'TFbn_{i}' for i in range(100)])
        tf_not_sexist_bigrams_dataframe = pd.concat([tf]).fillna(0)

        tf_test_dataframe = pd.concat([tf_sexist_dataframe,
                                      tf_not_sexist_dataframe,
                                      tf_sexist_bigrams_dataframe,
                                      tf_not_sexist_bigrams_dataframe], axis=1)
        X_test = tf_dataframe
        X_test['likes'] = likes_df
        X_test['dislikes'] = dislikes_df
        X_test['char-qty'] = char_qty_df
        X_test['word-qty'] = word_qty_df
        X_test['sexist'] = y_df
        X_test = X_test.fillna(0)
        X_test = X_test.sample(frac=1)
        X_test.to_csv(f'./data/{i+1}_test.csv', index=False)

def select_df_columns(df, columns):
    list_features = [df[df.columns[list(df.columns).index(x)]] for x in columns]
    X = pd.DataFrame(list_features).transpose()
    return X

def build_results_report(title,precision,recall,fscore,f1macro):
    report = {
        'title': title,
        'precision': {'1': [], '0': []},
        'recall': {'1': [], '0': []},
        'f1': {'1': [], '0': []},
        'f1_macro': []
    }
    report['precision']['1'].append(precision[0])
    report['recall']['1'].append(recall[0])
    report['f1']['1'].append(fscore[0])
    report['precision']['0'].append(precision[1])
    report['recall']['0'].append(recall[1])
    report['f1']['0'].append(fscore[1])
    report['f1_macro'].append(f1macro)
    return report

def print_report(report):
    print(f'>>>> {report["title"]} results')
    print('\t\t sexist \t not-sexist')
    print('precision\t %.5f \t %.5f' % (
        np.mean(report["precision"]["1"]), np.mean(report["precision"]["0"])))
    print('recall\t\t %.5f \t %.5f' %
          (np.mean(report["recall"]["1"]), np.mean(report["recall"]["0"])))
    print('f1\t\t %.5f \t %.5f' %
          (np.mean(report["f1"]["1"]), np.mean(report["f1"]["0"])))
    print('')

# Training and testing models

In [32]:
from sklearn.pipeline import make_pipeline
from joblib import parallel_backend
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

df = pd.read_csv(DATAFRAME_PATH)

if GENERATE_TRAIN_TEST:
    generate_cross_validation_train_and_test(df)

results = []
for i in tqdm(range(0, 10)):
    # ------- loading data
    train = pd.read_csv(f'./data/{i+1}_train.csv')
    test = pd.read_csv(f'./data/{i+1}_test.csv')
    X_train = train.drop(columns=['sexist'])
    y_train = train['sexist']
    X_test = test.drop(columns=['sexist'])
    y_test = test['sexist']
    
    # ------- selection features combination
    feature_combinations = {
        'TFus' : [x for x in list(train.columns) if 'TFus' in x], # sexist unigrams TFs
        'TFun' : [x for x in list(train.columns) if 'TFun' in x], # not-sexist unigrams TFs
        'TFbs' : [x for x in list(train.columns) if 'TFbs' in x], # sexist bigrams TFs
        'TFbn' : [x for x in list(train.columns) if 'TFbn' in x], # not sexist bigrams TFs
        'L' : ['likes'], # likes quantity
        'D' : ['dislikes'], # dislikes quantity
        'CQ' : ['char-qty'], # chars quantity
        'WQ' : ['word-qty'] # word quantity
    }
    feature_combinations['CQWQ'] = feature_combinations['CQ'] + feature_combinations['WQ']
    feature_combinations['LD'] = feature_combinations['L'] + feature_combinations['D']
    feature_combinations['Q'] = feature_combinations['LD'] + feature_combinations['CQWQ']
    
    feature_combinations['TFu'] = feature_combinations['TFus'] + feature_combinations['TFun']
    feature_combinations['TFb'] = feature_combinations['TFbs'] + feature_combinations['TFbn']
    feature_combinations['TF'] = feature_combinations['TFu'] + feature_combinations['TFb']
    feature_combinations['TFuQ'] = feature_combinations['TFu'] + feature_combinations['Q']    
    
    combination_results = dict({})

    for combination in tqdm(feature_combinations.keys()):
        X_train_ = select_df_columns(X_train, feature_combinations[combination])
        X_test_ = select_df_columns(X_test, feature_combinations[combination])

        # ------ training SVM
        model_path = f'{MODELS_PATH}/{i}_SVM_{combination}'

        try:
            svm = pickle.load(open(model_path, 'rb'))
        except:
            svm = SVC(kernel='rbf', C=1.0, gamma='scale')
            with parallel_backend('threading', n_jobs=12):
                svm.fit(X_train_ ,y_train)
            pickle.dump(svm, open(model_path, 'wb'))

        # ------ testing SVM
        y = svm.predict(X_test_)
        precision, recall, fscore, _ = score(y_test, y, average=None, labels=[1, 0])
        report = build_results_report(f'{i}_SVM_{combination}', precision, recall, fscore, f1_score(y_test,y))
        combination_results[f'{i}_SVM_{combination}'] = report
        # print_report(report)

        # ------ training KNN
        model_path = f'{MODELS_PATH}/{i}_KNN_{combination}'
        try:
            knn = pickle.load(open(model_path, 'rb'))
        except:
            knn = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='euclidean')
            with parallel_backend('threading', n_jobs=12):
                knn.fit(X_train_,y_train)
            pickle.dump(knn, open(model_path, 'wb'))

        # ------ testing KNN
        y = knn.predict(X_test_)
        precision, recall, fscore, _ = score(y_test, y, average=None, labels=[1, 0])
        report = build_results_report(f'{i}_KNN_{combination}', precision, recall, fscore, f1_score(y_test,y))
        combination_results[f'{i}_KNN_{combination}'] = report
        # print_report(report)

        # ------ training RFC
        model_path = f'{MODELS_PATH}/{i}_RFC_{combination}'
        print(model_path)
        try:
            rfc = pickle.load(open(model_path, 'rb'))
        except:
            rfc = RandomForestClassifier(n_estimators=200, max_depth=8)
            with parallel_backend('threading', n_jobs=12):
                rfc.fit(X_train_,y_train)
            pickle.dump(rfc, open(model_path, 'wb'))
        # ------ testing RFC
        y = rfc.predict(X_test_)
        precision, recall, fscore, _ = score(y_test, y, average=None, labels=[1, 0])
        report = build_results_report(f'{i}_RFC_{combination}', precision, recall, fscore, f1_score(y_test,y))
        combination_results[f'{i}_RFC_{combination}'] = report
        # print_report(report)

    results.append(combination_results)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

./data/models/0_RFC_TFus
./data/models/0_RFC_TFun
./data/models/0_RFC_TFbs
./data/models/0_RFC_TFbn
./data/models/0_RFC_L
./data/models/0_RFC_D
./data/models/0_RFC_CQ
./data/models/0_RFC_WQ
./data/models/0_RFC_CQWQ
./data/models/0_RFC_LD
./data/models/0_RFC_Q
./data/models/0_RFC_TFu
./data/models/0_RFC_TFb
./data/models/0_RFC_TF


/Users/braga/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/braga/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


./data/models/0_RFC_TFuQ


...

/Users/braga/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/braga/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


./data/models/9_RFC_TF


/Users/braga/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/braga/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/braga/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/braga/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.24.1 when using version 0

./data/models/9_RFC_TFuQ


/Users/braga/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/braga/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [33]:
pickle.dump(mean_results, open('./data/results','wb'))

# Computing results

In [39]:
import re
final_results = dict({})
mean_results = dict({})

for result in results:
    for key in result.keys():
        final_key = re.sub('\d+_', '', key)
        if final_key not in final_results.keys():    
            final_results[final_key] = {
                'precision': {'1': [], '0': []},
                'recall': {'1': [], '0': []},
                'f1': {'1': [], '0': []},
                'f1_macro': []
            }

        current = result[key]
        final_results[final_key]['precision']['1'].append(current['precision']['1'][0])
        final_results[final_key]['precision']['0'].append(current['precision']['0'][0])
        final_results[final_key]['recall']['1'].append(current['recall']['1'][0])
        final_results[final_key]['recall']['0'].append(current['recall']['0'][0])
        final_results[final_key]['f1']['1'].append(current['f1']['1'][0])
        final_results[final_key]['f1']['0'].append(current['f1']['0'][0])
        final_results[final_key]['f1_macro'].append(current['f1_macro'][0])

In [41]:
def med(array):
    return sum(array)/10

for key in final_results.keys():
    mean_results[key] = {
        'title': key,
        'precision': {'1': med(final_results[key]['precision']['1']),
                      '0': med(final_results[key]['precision']['0'])},
        'recall': {'1': med(final_results[key]['recall']['1']),
                      '0': med(final_results[key]['recall']['0'])},
        'f1': {'1': med(final_results[key]['f1']['1']),
                      '0': med(final_results[key]['f1']['0'])},
    }

In [42]:
for key in mean_results.keys():
    if 'SVM' in key:
        print_report(mean_results[key])
        
for key in mean_results.keys():
    if 'KNN' in key:
        print_report(mean_results[key])
        
for key in mean_results.keys():
    if 'RFC' in key:
        print_report(mean_results[key])

>>>> SVM_TFus results
		 sexist 	 not-sexist
precision	 0.95534 	 0.81824
recall		 0.80631 	 0.95855
f1		 0.87452 	 0.88285

>>>> SVM_TFun results
		 sexist 	 not-sexist
precision	 0.92205 	 0.78447
recall		 0.76791 	 0.92863
f1		 0.83795 	 0.85048

>>>> SVM_TFbs results
		 sexist 	 not-sexist
precision	 0.77546 	 0.55670
recall		 0.35829 	 0.88595
f1		 0.49012 	 0.68376

>>>> SVM_TFbn results
		 sexist 	 not-sexist
precision	 0.59838 	 0.79331
recall		 0.92471 	 0.31770
f1		 0.72659 	 0.45370

>>>> SVM_L results
		 sexist 	 not-sexist
precision	 0.58112 	 0.52410
recall		 0.50374 	 0.60082
f1		 0.53967 	 0.55985

>>>> SVM_D results
		 sexist 	 not-sexist
precision	 0.62412 	 0.50791
recall		 0.28503 	 0.81129
f1		 0.39134 	 0.62472

>>>> SVM_CQ results
		 sexist 	 not-sexist
precision	 0.53928 	 0.53557
recall		 0.81497 	 0.23457
f1		 0.64906 	 0.32625

>>>> SVM_WQ results
		 sexist 	 not-sexist
precision	 0.53924 	 0.54124
recall		 0.83048 	 0.21987
f1		 0.65389 	 0.31271

>>>> SVM_C

# Wilcoxon

In [77]:
final_results['BERT']= {'f1_macro': [
    0.571069182389937,
    0.45852187028657615,
    0.5364583333333334,
    0.5258964143426295,
    0.5313351498637601,
    0.4828571428571429,
    0.43333333333333335,
    0.5463659147869674,
    0.5493333333333332,
    0.5195822454308094
]}

final_results['B_SVM'] = {'f1_macro': [0.58, 0.6, 0.6, 0.6, 0.59, 0.59, 0.6, 0.6, 0.59, 0.6]}
final_results['B_LR']= {'f1_macro': [0.63, 0.63, 0.63, 0.63, 0.63, 0.63, 0.63, 0.63, 0.63, 0.63]}

tests = dict({})
from scipy.stats import wilcoxon
for key_1 in final_results.keys():
    for key_2 in final_results.keys():
        if (key_1 != key_2) and \
            f'{key_1} x {key_2}' not in tests.keys() and\
            f'{key_2} x {key_1}' not in tests.keys():
            values_y = final_results[key_1]['f1_macro']
            values_x = final_results[key_2]['f1_macro']
            test_stat, test_p = wilcoxon(values_y, values_x)
            tests[f'{key_1} x {key_2}'] = test_p

In [80]:
pickle.dump(tests, open('./data/wilcoxon', 'wb'))